![cropped-SummerWorkshop_Header.png](resources/banner.jpg)

<h1 align="center">Allen Brain Observatory Visual Behavior Neuropixels </h1> 
<h2 align="center"> SWDB 2024 - Day 1 </h2> 
<h3 align="center"> Afternoon Session </h3> 

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
**Databook**

The databook is your one-stop-shop for understanding the various dimensions of this dataset, the methods used, and how to access the data that you are interested in. 

You can find the pages for the Visual Behavior Ophys dataset here: https://allenswdb.github.io/physiology/ephys/visual-behavior/VB-Neuropixels.html

![vbo_databook.png](resources/databook_vbnp.png)

</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorNeuropixelsProjectCache

pd.set_option('display.max_columns', None)
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import platform
platstring = platform.platform()

if ('Darwin' in platstring) or ('macOS' in platstring):
    # macOS 
    data_root = "/Volumes/Brain2024/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on CodeOcean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2024/"

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
# Outline

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## VBN data structures
#### 1. VBN cache object
#### 2. "Sessions" table
#### 3. "Units" table
#### 4. Session object
#### 5. "Stimulus" table
#### 6. "Trials" table

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
# 1. VBN cache object

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
* Access to metadata tables describing the entire dataset
* Tools for downloading data (as NWB files)
* Tools for instantiating "Session" objects from NWB files

</div>

In [ ]:
cache = VisualBehaviorNeuropixelsProjectCache.from_local_cache(cache_dir=data_root, use_static_cache=True)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
# 2. "Sessions" table

</div>

In [ ]:
sessions = cache.get_ecephys_session_table()

In [ ]:
sessions.head()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## How many experimental sessions in dataset?

</div>

In [ ]:
len(sessions)

In [ ]:
sessions.genotype.value_counts()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## How many units recorded per experiment?

</div>

In [ ]:
plt.hist(sessions.unit_count)
plt.xlabel('# of units recorded per session')
plt.ylabel('# of sessions')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
# 3. "Units" table

</div>

In [ ]:
units = cache.get_unit_table()

In [ ]:
units.head()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## How many units are in dataset?

</div>

In [ ]:
units.shape

In [ ]:
# Select 'good' quality units
units = units[units.quality=='good']

In [ ]:
units.shape

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## Which brain regions are recorded?

</div>

In [ ]:
regions = units.structure_acronym.value_counts()
regions

In [ ]:
regions[:25].plot.bar()
plt.xlabel('Brain regions')
plt.ylabel('# of units')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## Select units in visual cortex

</div>

In [ ]:
v_units = units[units.structure_acronym.str.contains('VIS')]   
v_units.shape

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## Select Broad and Narrow spiking units
#### aka Regular and Fast spiking units

</div>

In [ ]:
plt.hist(v_units.waveform_duration,bins=25,range=(0,1));

plt.xlabel("spike duration (ms)")
plt.ylabel("# of units in visual cortex")

In [ ]:
fast_spiking_units = v_units[v_units.waveform_duration < 0.4]

In [ ]:
plt.hist(fast_spiking_units.waveform_duration,bins=25,range=(0,1));

plt.xlabel("spike duration (ms)")
plt.ylabel("# of units in visual cortex")

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
# . . . 

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
# 4. "Session" object: data from single experiment

</div>

In [ ]:
session_id = 1053941483

In [ ]:
session = cache.get_ecephys_session(session_id)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## Get units table for this one session

</div>

In [ ]:
units = session.get_units()
units.head(2)

In [ ]:
# Get channels table -- this tells us the location of every electrode channel
channels = session.get_channels()
channels.head(2)

In [ ]:
# Merge channels table into units table using 
units = units.merge(channels, left_on='peak_channel_id', right_index=True)
units.head(2)

In [ ]:
# Select units that are in visual cortex
v_units = units[units.structure_acronym.str.contains('VIS')]

In [ ]:
v_units.structure_acronym.value_counts().plot.bar();

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## Make raster plot of one unit's spike train

</div>

In [ ]:
# Example unit
example_unit_id = v_units.index[27]
example_unit_id

In [ ]:
# Get spike times for example unit
example_unit_spike_times = session.spike_times[example_unit_id]
example_unit_spike_times.shape

In [ ]:
# Plot raster
plt.eventplot(example_unit_spike_times)
plt.xlabel('Seconds');

In [ ]:
# Make raster for shorter segment of time
raster_start_time = 407
raster_end_time = 427

plt.figure(figsize=(11,1))
plt.eventplot(example_unit_spike_times,linewidth=0.75)
plt.xlim(raster_start_time,raster_end_time)
plt.xlabel('Seconds');

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## 5. "Stimulus" table

</div>

In [ ]:
stim_table = session.stimulus_presentations

In [ ]:
stim_table.head()

In [ ]:
# How many stimulus presentations in session?
len(stim_table)

In [ ]:
# Select stimuli presented during time period in raster plot
stim_times = stim_table[(stim_table.start_time>raster_start_time) & (stim_table.start_time<raster_end_time)]
stim_times.head()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

## Plot stimulus presentations with raster plot

</div>

In [ ]:
# What are the unique stimulus image names
unique_stim = stim_times.image_name.unique()
print(unique_stim)

In [ ]:
# Map color to each image
colors = ['blue','green','purple','gray']
color_stim_dict = dict(zip(unique_stim,colors))
color_stim_dict

In [ ]:
plt.figure(figsize=(11,1))
plt.eventplot(example_unit_spike_times,linewidth=0.75)

plt.xlim(raster_start_time,raster_end_time)
plt.xlabel('Seconds');

for row in stim_times.iterrows():
    t_start = row[1].start_time
    plt.axvspan(t_start,t_start+0.25,alpha=0.3,color=color_stim_dict[row[1].image_name],lw=0)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## 6. Behavioral "Trials" table

</div>

In [ ]:
trials = session.trials
trials.head()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
   
## Plot behavior trial data on raster plot

</div>

In [ ]:
# Filter trials that occur during time window in our raster plot above
trials_for_plot = trials[(trials.start_time>raster_start_time) & (trials.start_time<raster_end_time)]
trials_for_plot

In [ ]:
# Get lick times
lick_times = trials_for_plot.lick_times.values
lick_times = np.concatenate(lick_times) # Flatten array of arrays to 1D array
lick_times

In [ ]:
# Get reward times
reward_times = trials_for_plot.reward_time.values
reward_times

In [ ]:
plt.figure(figsize=(11,1))

# Plot spike train
plt.eventplot(example_unit_spike_times,linewidth=0.75)

# Plot stimulus times
for row in stim_times.iterrows():
    t_start = row[1].start_time
    plt.axvspan(t_start,t_start+0.25,alpha=0.3,color=color_stim_dict[row[1].image_name],lw=0)

# Plot lick and reward times
plt.eventplot(lick_times,lineoffsets=2.5,color='black',linewidth=1)
plt.plot(reward_times,np.ones_like(reward_times)*2.5,color='blue',marker='o',linestyle='');

# Formatting
plt.xlim(raster_start_time,raster_end_time)
plt.xlabel('Seconds');

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

***Homework 1***

* Working with the example unit above, compute the trial-averaged spike rate in response to each of the 8 images.
* Plot with error bars.

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

***Homework 2***

* Make raster plots showing the spiking of this unit on each hit versus each miss trial for one of the images.
* Is the unit’s response different on hit versus miss trials? If so, why might this be the case?
* Does hit versus miss activity vary for the preferred versus non-preferred image?

</div>